In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
#export
from collections import OrderedDict
from typing import Tuple, Union

from torch.utils.data import Dataset, DataLoader

from loop.config import defaults
from loop.utils import broadcast

In [ ]:
#export
class NamedList:
    """A convenience wrapper that allows to iterate over ordered dict values and
    access its elements using numerical indicies.
    """
    def __init__(self, od: OrderedDict):
        self.od = OrderedDict(od)
        self.names = list(self.od)
        
    def __iter__(self):
        self.index = -1
        return self
    
    def __next__(self):
        self.index += 1
        if self.index >= len(self):
            raise StopIteration()
        return self[self.index]
    
    def __len__(self): 
        return len(self.od)
    
    def __getitem__(self, item: Union[str, int]):
        if isinstance(item, str):
            return self.od[item]
        elif isinstance(item, int):
            return self.od[self.names[item]]
        return TypeError(f'invalid index type: {type(item)}')

In [ ]:
#export
class Phase:
    """Model training loop phase.

    Each model's training loop iteration could be separated into (at least) two
    phases: training and validation. The instances of this class track metrics and 
    counters, related to the specific phase, and keep the reference to subset of 
    data, used during phase.
    """
    def __init__(self, name: str, loader: DataLoader, grad: bool=True):
        self.name = name
        self.loader = loader
        self.grad = grad
        self.batch_loss = None
        self.batch_index = 0
        self.rolling_loss = 0
        self.losses = []
        self.metrics = OrderedDict()

    @property
    def last_loss(self):
        return self.losses[-1] if self.losses else None

    @property
    def last_metrics(self):
        metrics = OrderedDict()
        metrics[f'{self.name}_loss'] = self.last_loss
        for name, values in self.metrics.items():
            metrics[f'{self.name}_{name}'] = values[-1]
        return metrics

    @property
    def metrics_history(self):
        metrics = OrderedDict()
        for name, values in self.metrics.items():
            metrics[f'{self.name}_{name}'] = values
        return metrics
    
    def get_last_value(self, metric):
        return self.last_metrics[f'{self.name}_{metric}']

    def update(self, loss: float):
        self.losses.append(loss)

    def update_metric(self, name: str, value: 'scalar'):
        if name not in self.metrics:
            self.metrics[name] = []
        self.metrics[name].append(value)
        
    @staticmethod
    def make_train_valid(trn_ds: Dataset, val_ds: Dataset, 
                         bs: int=defaults.bs,
                         num_workers: Union[Tuple, int]=0):
        """Creates two loop's phases, train and valid.
        
        The phases are thin wrappers on top of data loaders intended to track additional 
        information gathered during model's fitting process, like loss, performance 
        metrics, etc.
        """
        trn, val = broadcast(num_workers, 2)
        phs = OrderedDict()
        phs['train'] = Phase('train', DataLoader(trn_ds, bs, shuffle=True, num_workers=trn))
        phs['valid'] = Phase('valid', DataLoader(val_ds, bs, num_workers=val), grad=False)
        return NamedList(phs)

In [ ]:
nl = NamedList([
    ('first', 1),
    ('second', 2),
    ('third', 3)
])

In [ ]:
assert nl[0] == nl['first']
assert nl[1] == nl['second']
assert nl[2] == nl['third']
assert len(nl) == 3
assert list(nl) == [1, 2, 3]

In [ ]:
from torchvision.datasets import MNIST

root = defaults.datasets/'mnist'
trn_ds = MNIST(root, train=True)
val_ds = MNIST(root, train=False)
phases = Phase.make_train_valid(trn_ds, val_ds)

assert len(phases) == 2
assert phases['train'].loader is not None
assert phases['valid'].loader is not None
assert phases['train'].grad 
assert not phases['valid'].grad